In [12]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
from torch.autograd import Variable

import os
import csv
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload

%autoreload 2
plt.ion()   # interactive mode

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# Prepare the Dataset
IMAGE_DIR = "Dataset/2d_images/"
MASK_DIR = "Dataset/2d_masks/"
with open('Dataset/Lung_CT_Dataset.csv','w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['filename', 'mask'])
    
    for p in os.listdir(IMAGE_DIR):
        image_path = os.path.join(IMAGE_DIR, p)
        mask_path = os.path.join(MASK_DIR, p)
        writer.writerow([image_path, mask_path])


In [14]:
data = pd.read_csv("Dataset/Lung_CT_Dataset.csv")
data = data.iloc[np.random.permutation(len(data))]
p = int(len(data)*0.7)
train, validation = data[:p], data[p:]
train.to_csv("Dataset/Lung_CT_Train.csv", index=False)
validation.to_csv("Dataset/Lung_CT_Validation.csv", index=False)

In [15]:
from dataset import LungCTDataset
lung_ct_train_dataset = LungCTDataset(csv_file='Dataset/Lung_CT_Train.csv', root_dir='./')
lung_ct_val_dataset = LungCTDataset(csv_file='Dataset/Lung_CT_Validation.csv', root_dir='./')
train_dataloader = DataLoader(lung_ct_train_dataset, batch_size=100, shuffle=True, num_workers=4)
val_dataloader = DataLoader(lung_ct_val_dataset, batch_size=100, shuffle=True, num_workers=4)

In [16]:
from model import * 
model_instance = UNet(1, 1)

In [17]:
# model_instance.apply(weights_init)

In [18]:
optimizer = optim.Adam(model_instance.parameters(), lr=0.000001)
criterion = nn.BCELoss()

In [19]:
def train(model, epoch):
    model.train()
    correct = 0
    for batch_idx, data in enumerate(train_dataloader):
        data, target = Variable(data["image"]), Variable(data["mask"])
        optimizer.zero_grad()
        output = model.forward(data.float())
        loss = criterion(output.float(), target.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % 1 == 0:
            #print(loss.data)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_dataloader.dataset),
                100. * batch_idx / len(train_dataloader), loss.data)) # sangkny [0]
    

In [20]:
def test(model):
    model.eval()
    test_loss = 0
    for data in val_dataloader:
        data, target = Variable(data["image"], volatile=True), Variable(data["mask"])
        output = model(data.float())
        # print(output.data[0]) # need to change output.data because of version changes
        test_loss += criterion(output.float(), target.float()).data # [0] # sum up batch loss
    test_loss /= len(val_dataloader.dataset)
    print("Average Loss: ", test_loss)


In [21]:
# 99 % validation accuracy 
for epoch in range(1, 15):
    train(model_instance, epoch)
    test(model_instance)    

Train Epoch: 1 [0/185 (0%)]	Loss: 0.892298
Train Epoch: 1 [85/185 (50%)]	Loss: 0.901305
Average Loss:  tensor(0.0069)
Train Epoch: 2 [0/185 (0%)]	Loss: 0.900302
Train Epoch: 2 [85/185 (50%)]	Loss: 0.890575
Average Loss:  tensor(0.0084)
Train Epoch: 3 [0/185 (0%)]	Loss: 0.893602
Train Epoch: 3 [85/185 (50%)]	Loss: 0.903235
Average Loss:  tensor(0.0091)
Train Epoch: 4 [0/185 (0%)]	Loss: 0.892886
Train Epoch: 4 [85/185 (50%)]	Loss: 0.900117
Average Loss:  tensor(0.0097)
Train Epoch: 5 [0/185 (0%)]	Loss: 0.892075
Train Epoch: 5 [85/185 (50%)]	Loss: 0.898867
Average Loss:  tensor(0.0101)
Train Epoch: 6 [0/185 (0%)]	Loss: 0.891311
Train Epoch: 6 [85/185 (50%)]	Loss: 0.900516
Average Loss:  tensor(0.0104)
Train Epoch: 7 [0/185 (0%)]	Loss: 0.899841
Train Epoch: 7 [85/185 (50%)]	Loss: 0.892044
Average Loss:  tensor(0.0110)
Train Epoch: 8 [0/185 (0%)]	Loss: 0.900197
Train Epoch: 8 [85/185 (50%)]	Loss: 0.890548
Average Loss:  tensor(0.0116)
Train Epoch: 9 [0/185 (0%)]	Loss: 0.891941
Train Epoch: 

In [22]:
torch.save(model_instance, './saved_models/mini_unet1.pth')